# Testing Coordinator

In [1]:
# logging imports
import logging
from logging import StreamHandler

# essential kruppe imports
from kruppe.llm import OpenAILLM

# toolkit import
from kruppe.llm import OpenAIEmbeddingModel
from kruppe.functional.docstore.mongo_store import MongoDBStore
from kruppe.functional.rag.vectorstore.chroma import ChromaVectorStore
from kruppe.functional.rag.index.vectorstore_index import VectorStoreIndex
from kruppe.functional.rag.retriever.simple_retriever import SimpleRetriever
from kruppe.functional.ragquery import RagQuery
from kruppe.functional.llmquery import LLMQuery
from kruppe.functional.newshub import NewsHub
from kruppe.functional.finhub import FinHub
from kruppe.data_source.news.nyt import NewYorkTimesData
from kruppe.data_source.news.ft import FinancialTimesData
from kruppe.data_source.news.newsapi import NewsAPIData
from kruppe.data_source.finance.yfin import YFinanceData

# researcher import
from kruppe.algorithm.librarian import Librarian
from kruppe.algorithm.coordinator import Coordinator

In [2]:
# set up logging

# handlers

# set up logging for jupyter notebook
ch = StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
ch.setLevel(logging.INFO)

# set up logging for everything
log_file_path = "/Users/danielliu/Workspace/fin-rag/logs/everything.log"
with open(log_file_path, 'w') as f:
    pass

fh_all = logging.FileHandler(log_file_path)
fh_all.setFormatter(formatter)
fh_all.setLevel(logging.DEBUG)

# set up logging for llm
log_file_path = "/Users/danielliu/Workspace/fin-rag/logs/llm.log"
with open(log_file_path, 'w') as f:
    pass

fh_llm = logging.FileHandler(log_file_path)
fh_llm.setFormatter(formatter)
fh_llm.setLevel(logging.DEBUG)

# set up logging for scraper
log_file_path = "/Users/danielliu/Workspace/fin-rag/logs/scraper.log"
with open(log_file_path, 'w') as f:
    pass

fh_scraper = logging.FileHandler(log_file_path)
fh_scraper.setFormatter(formatter)
fh_scraper.setLevel(logging.DEBUG)

# set up loggers
root_logger = logging.getLogger()
root_logger.setLevel(logging.ERROR)
root_logger.handlers.clear()
root_logger.addHandler(fh_all) # log everything to a file

kruppe_logger = logging.getLogger("kruppe.algorithm")
kruppe_logger.setLevel(logging.INFO)
kruppe_logger.addHandler(ch) # log to console

logger = logging.getLogger("kruppe.llm")
logger.setLevel(logging.DEBUG)
logger.addHandler(fh_llm) # log llm output to a file instead of console.
logger.propagate = False # prevent logging from propagating to the root logger

logger_scraper = logging.getLogger("kruppe.data_source.scraper")
logger_scraper.setLevel(logging.DEBUG)
logger_scraper.addHandler(fh_scraper) # log scraper output to a file instead of console.
logger_scraper.propagate = False # prevent logging from propagating to the console

In [3]:
reset_db=False

db_name = "kruppe_librarian"
collection_name = "general_news_04_20_2025"

# Create doc store
unique_indices = [['title', 'datasource']] # NOTE: this is important to avoid duplicates
docstore = await MongoDBStore.acreate_db(
    db_name=db_name,
    collection_name=collection_name,
    unique_indices=unique_indices,
    reset_db=reset_db
)

# Create vectorstore index
embedding_model = OpenAIEmbeddingModel()
vectorstore = ChromaVectorStore(
    embedding_model=embedding_model,
    collection_name=collection_name,
    persist_path='/Volumes/Lexar/Daniel Liu/vectorstores/kruppe_librarian'
)

index = VectorStoreIndex(vectorstore=vectorstore)
retriever = SimpleRetriever(index=index)

if reset_db:
    vectorstore.clear()

# vectorstore.clear()
# docs = await docstore.aget_all_documents()
# print(len(docs))
# await index.async_add_documents(docs)


In [4]:
print("Number of documents:", docstore.size())
print("Number of chunks:", vectorstore.size())

Number of documents: 1521
Number of chunks: 12388


In [5]:
rag_query_engine = RagQuery(
    retriever = retriever,
    llm = OpenAILLM()
)

llm_query_engine = LLMQuery(
    llm = OpenAILLM()
)

news_hub = NewsHub(news_sources=[
    NewYorkTimesData(headers_path="../../.nyt-headers.json"),
    FinancialTimesData(headers_path="../../.ft-headers.json"),
    NewsAPIData()
])

fin_hub = FinHub(
    fin_source = YFinanceData(),
    llm = OpenAILLM()
)

In [6]:
toolkit_librarian = [
    rag_query_engine.rag_query,
    llm_query_engine.llm_query,
    news_hub.news_search,
    news_hub.news_recent,
    news_hub.news_archive,
    fin_hub.get_company_background,
    fin_hub.get_company_income_stmt,
    fin_hub.get_company_balance_sheet,
    fin_hub.analyze_company_financial_stmts
]

toolkit_researcher = [
    rag_query_engine.rag_query,
    llm_query_engine.llm_query,
    news_hub.news_search,
    # news_hub.news_recent,
    # news_hub.news_archive,
    fin_hub.get_company_background,
    fin_hub.get_company_income_stmt,
    fin_hub.get_company_balance_sheet,
    fin_hub.analyze_company_financial_stmts
]

# Coordinator

In [7]:
query = "How has the company Everest's performance changed over the last year? (2025)"

In [8]:
llm = OpenAILLM(model="gpt-4.1-mini")
librarian = Librarian(
    llm=llm,
    toolkit=toolkit_librarian,
    docstore=docstore,
    index=index,
    max_steps=20
)

In [9]:
tree_configs = {
    "llm": OpenAILLM(),
    "toolkit": toolkit_researcher,
    "docstore": docstore,
    "index": index,
    "max_step": 10,
    "max_degree": 2
}

In [10]:
coordinator = Coordinator(
    llm=llm,
    tree_configs = tree_configs,
    librarian=librarian,
)

### Generate Domain Experts

In [11]:
# experts = await coordinator.generate_domain_experts(query)
# experts

In [12]:
# filtered_experts = await coordinator.filter_domain_experts(query, experts, 3)
# filtered_experts

### Generate Background

In [13]:
# bkg_report = await coordinator.generate_background(query)

In [14]:
# print(bkg_report.text)

### Execute

In [15]:
reports = await coordinator.execute(query, 2)

Thinking (step 1)
Tool call: get_company_background ({"ticker":"EVEREST"})
Thinking (step 2)
Tool call: llm_query ({"query":"Everest company, business profile, stock ticker, or related information"})
Thinking (step 3)
Tool call: get_company_background ({"ticker":"RE"})
Thinking (step 4)
Tool call: news_archive ({"start_date":"2025-01-01","end_date":"2025-04-23","max_results":10,"keywords":"Everest Re Group, RE"})
Added 1 documents to index and docstore (out of 10 total documents)
Thinking (step 5)
Tool call: news_recent ({"days":120,"max_results":10,"keywords":"Everest Re Group"})
Added 10 documents to index and docstore (out of 10 total documents)
Thinking (step 6)
Tool call: get_company_income_stmt ({"ticker": "RE", "years": 2})
Thinking (step 7)
Tool call: get_company_balance_sheet ({"ticker":"RE","years":2})
Thinking (step 8)
Tool call: llm_query ({"query":"Everest Re Group financial performance and trends in 2025, including revenues, profits, and major financial metrics analysis"}

In [16]:
for report in reports:
    print("EXPERT:", report.metadata['expert'])
    print("EXPERT DESCRIPTION:", report.metadata['expert_description'])
    print(report)
    print('-'*50)

EXPERT: Financial Analyst
EXPERT DESCRIPTION: The Financial Analyst is an expert in financial statements, market data, and performance metrics, specializing in analyzing company performance over time, investment viability, and shareholder value.
Final Report:

Restated Working Hypothesis: Everest Re Group’s performance in 2025 likely demonstrated moderate changes consistent with industry-wide trends in the reinsurance sector, influenced primarily by underwriting outcomes and investment returns, with no material impact from company-specific events since none were publicly reported.

Summary and Analysis:

The investigation began with the premise that Everest Re Group’s 2025 financial performance caused only moderate changes reflecting broader reinsurance industry trends. Critical drivers were assumed to be underwriting results and investment income, with no significant company-specific disruptions documented publicly.

Initial research efforts to locate Everest Re Group’s 2025 or interi

In [18]:
summary_report = await coordinator.summarize_reports()
summary_report

Response(text='The provided research reports share a common challenge: the absence of direct, publicly available 2025 financial disclosures from Everest Re Group. Consequently, all analyses rely substantially on industry trends, peer performance, partial market data, and reasoned inference rather than detailed company-specific figures. Despite this limitation, several consistent themes and some nuanced differences emerge across the reports.\n\n---\n\n### 1. **Moderate Performance Change Consistent with Industry Trends**\n\n**Key Findings:**\n- Everest Re’s 2025 financial performance likely saw only moderate year-over-year changes.\n- These changes predominantly reflect broader reinsurance sector conditions rather than company-specific disruptions.\n- Industry-wide pressures included elevated underwriting expenses caused by substantial natural catastrophe claims.\n- Investment income likely provided partial offsetting gains due to rising interest rates improving asset returns.\n- No mat

In [20]:
print(summary_report.text)

The provided research reports share a common challenge: the absence of direct, publicly available 2025 financial disclosures from Everest Re Group. Consequently, all analyses rely substantially on industry trends, peer performance, partial market data, and reasoned inference rather than detailed company-specific figures. Despite this limitation, several consistent themes and some nuanced differences emerge across the reports.

---

### 1. **Moderate Performance Change Consistent with Industry Trends**

**Key Findings:**
- Everest Re’s 2025 financial performance likely saw only moderate year-over-year changes.
- These changes predominantly reflect broader reinsurance sector conditions rather than company-specific disruptions.
- Industry-wide pressures included elevated underwriting expenses caused by substantial natural catastrophe claims.
- Investment income likely provided partial offsetting gains due to rising interest rates improving asset returns.
- No material company-specific eve

In [17]:
# for hyp_researcher in coordinator._research_forest:
#     for report in hyp_researcher.research_reports:
#         print("EXPERT:", report.metadata['expert'])
#         print("EXPERT DESCRIPTION:", report.metadata['expert_description'])
#         print(report)
#         print('-'*50)